In [20]:
import numpy as np 
import pandas as pd
import matplotlib as plt
import seaborn as sns
import requests
import json
from bs4 import BeautifulSoup

In [21]:
df = pd.read_csv('data/pgaTourData.csv')

In [22]:
df.head()

,Player Name,Rounds,Fairway Percentage,Year,Avg Distance,gir,Average Putts,Average Scrambling,Average Score,Points,Wins,Top 10,Average SG Putts,Average SG Total,SG:OTT,SG:APR,SG:ARG,Money
0,Henrik Stenson,60.0,75.19,2018,291.5,73.51,29.93,60.67,69.617,868,NaN,5.0,-0.207,1.153,0.427,0.960,-0.027,"$2,680,487"
1,Ryan Armour,109.0,73.58,2018,283.5,68.22,29.31,60.13,70.758,"1,006",1.0,3.0,-0.058,0.337,-0.012,0.213,0.194,"$2,485,203"
2,Chez Reavie,93.0,72.24,2018,286.5,68.67,29.12,62.27,70.432,"1,020",NaN,3.0,0.192,0.674,0.183,0.437,-0.137,"$2,700,018"
3,Ryan Moore,78.0,71.94,2018,289.2,68.80,29.17,64.16,70.015,795,NaN,5.0,-0.271,0.941,0.406,0.532,0.273,"$1,986,608"
4,Brian Stuard,103.0,71.44,2018,278.9,67.12,29.11,59.23,71.038,421,NaN,3.0,0.164,0.062,-0.227,0.099,0.026,"$1,089,763"


#### Although the dataset seems pretty clean, it still  contains some null error that we want to get rid of, plus we only want to keep the data for the years of 2017 and 2018. Moreover, and for greater simplicity I´m going to rename most headers in order to replace spaces between words for '_' . 
#### So... let´s do some cleaning!!

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1678 entries, 0 to 1677
Data columns (total 18 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Player_Name         1678 non-null   object 
 1   Rounds              1678 non-null   int64  
 2   Fairway_Percentage  1678 non-null   float64
 3   Year                1678 non-null   int64  
 4   Avg_Distance        1678 non-null   float64
 5   gir                 1678 non-null   float64
 6   Average_Putts       1678 non-null   float64
 7   Average_Scrambling  1678 non-null   float64
 8   Average_Score       1678 non-null   float64
 9   Points              1674 non-null   object 
 10  Wins                1678 non-null   int64  
 11  Top_10              1678 non-null   int64  
 12  Average_SG_Putts    1678 non-null   float64
 13  Average_SG_Total    1678 non-null   float64
 14  SG_OTT              1678 non-null   float64
 15  SG_APR              1678 non-null   float64
 16  SG_ARG

In [23]:
df = df.dropna(axis=0, subset=['Rounds'])

In [24]:
df = df.rename(columns = {'Player Name': 'Player_Name', 'Fairway Percentage':'Fairway_Percentage', 'Avg Distance':'Avg_Distance', 'Average Putts':'Average_Putts', 'Average Scrambling': 'Average_Scrambling', 'Average Score':'Average_Score', 'Top 10':'Top_10', 'Average SG Putts':'Average_SG_Putts', 'Average SG Total': 'Average_SG_Total', 'SG:OTT': 'SG_OTT', 'SG:APR': 'SG_APR', 'SG:ARG':'SG_ARG'})

In [31]:
# Replace NaN for 0 in Top_10 and make it an int
df['Top_10'].fillna(0, inplace = True)
df['Top_10'] = df['Top_10'].astype(int)

In [32]:
# Replace NaN for 0 in Wins and make it an int
df['Wins'].fillna(0, inplace = True)
df['Wins'] = df['Wins'].astype(int)

In [33]:
# Make Rounds an int
df['Rounds'] = df['Rounds'].astype(int)

In [35]:
# Make Points an int

df.dropna(axis = 0, inplace = True )
df['Points'] = df['Points'].apply(lambda x: x.replace(',',''))
#df['Points'] = df['Points'].str.replace(',', '')
df['Points'] = df['Points'].astype(int)

In [36]:
# Perfect money column
df['Money'] = df['Money'].apply(lambda x: x.replace('$', ''))
df['Money'] = df['Money'].apply(lambda x: x.replace(',', ''))
df['Money'] = df['Money'].astype(int)

In [37]:
indexYears = df[df['Year'] <= 2016].index

In [38]:
df.drop(indexYears , inplace=True)

In [39]:
# This if finally how our clean dataset looks now.
df.head()

,Player_Name,Rounds,Fairway_Percentage,Year,Avg_Distance,gir,Average_Putts,Average_Scrambling,Average_Score,Points,Wins,Top_10,Average_SG_Putts,Average_SG_Total,SG_OTT,SG_APR,SG_ARG,Money
0,Henrik Stenson,60,75.19,2018,291.5,73.51,29.93,60.67,69.617,868,0,5,-0.207,1.153,0.427,0.960,-0.027,2680487
1,Ryan Armour,109,73.58,2018,283.5,68.22,29.31,60.13,70.758,1006,1,3,-0.058,0.337,-0.012,0.213,0.194,2485203
2,Chez Reavie,93,72.24,2018,286.5,68.67,29.12,62.27,70.432,1020,0,3,0.192,0.674,0.183,0.437,-0.137,2700018
3,Ryan Moore,78,71.94,2018,289.2,68.80,29.17,64.16,70.015,795,0,5,-0.271,0.941,0.406,0.532,0.273,1986608
4,Brian Stuard,103,71.44,2018,278.9,67.12,29.11,59.23,71.038,421,0,3,0.164,0.062,-0.227,0.099,0.026,1089763


In [41]:
df.to_csv('golf_data.csv')

### Now let´s continue with this dataset on another jupyter notebook...